<a href="https://colab.research.google.com/github/delhian/NLP_course/blob/master/Home%20Tasks/gpt_2_text_clf_unresolved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Возможности и ограничения языковых моделей

**Тут есть важное действие, которое нужно сделать перед тем, как запускать этот ноутбук**

Если вы делаете это задание в Google Colab, первым делом переключите Runtime в GPU. Это задание нормально посчитается и на CPU, но некоторые из будущих кейсов потребуют GPU (либо вам придётся несколько часов или даже дней, чтобы модель обучилась - и мы не преувиличиваем). Ещё мы рекомендуем переключить язык интерфейса на английский, потому что русская локализация ужасна да и вообще не нужна.

О том, как переключить рантайм: https://www.geeksforgeeks.org/how-to-use-google-colab/

In [1]:
!pip install datasets transformers[torch]==2.10.0 tqdm

     |████████████████████████████████| 287 kB 5.2 MB/s 
     |████████████████████████████████| 660 kB 37.9 MB/s 
     |████████████████████████████████| 1.2 MB 33.0 MB/s 
     |████████████████████████████████| 5.6 MB 36.1 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
     |████████████████████████████████| 125 kB 47.7 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 243 kB 46.4 MB/s 
     |████████████████████████████████| 1.3 MB 29.3 MB/s 
     |████████████████████████████████| 160 kB 47.9 MB/s 
     |████████████████████████████████| 271 kB 40.0 MB/s 


In [2]:
from transformers import AutoModel, AutoTokenizer, GPT2LMHeadModel
import datasets

import torch
from torch.nn import functional as F
from tqdm import tqdm
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
import transformers
print(transformers.__version__)  # should be above or equal 4.0.1

2.10.0


В этом модуле мы с вами познакомились с большими языковыми моделями и обсудили их возможности, а также ограничения. В этом задании вам предлагается поэкспериментировать с одной из больших языковых моделей -- GPT-2 -- и самим убедиться, так ли она хороша, как о ней рассказывают.

## Библиотека transformers

Вы уже сталкивались с библиотекой transformers в этом курсе.

Мы [загрузим gpt-2](https://huggingface.co/transformers/model_doc/gpt2.html) с помощью метода from_pretrained. Так как мы будем использовать её для задачи языкового моделирования, нам потребуется GPT2LMHeadModel, прочитать про неё можно [тут](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel). У GPT-2 свой токенизатор, про него можно почитать [тут](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2tokenizer).

GPT-2 -- большая модель, время ее работы на CPU будет слишком большим, так что мы сразу отправим её на GPU.

In [5]:
model_name = 'gpt2-medium'
device = 'cuda'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [7]:
model = GPT2LMHeadModel.from_pretrained(model_name)
model = model.to(device)

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

### Подсчёт числа параметров модели

Чтобы понять, насколько GPT большая модель, подсчитаем число ее параметров

In [10]:
# TASK: compute number of model parameters
# iterate over model weights and count number of parameters in each of them, then sum it up
# note: you may find model.parameters() method useful
# Our implementation is 2 lines

params = sum([params.numel() for params in model.parameters() ])
# YOUR CODE STARTS

# YOUR CODE ENDS
params

354823168

Как видим, это большое число параметров, на несколько порядок больше, чем те модели, которые вы обучали до этого! Использовать такое в продакшене как правило невозможно из-за больших требований по ресурсам

# Часть 1. Генерация отзывов моделью GPT

В преыдущем задании мы генерировали ровно 1 токен, так как нам нужно было предсказать сентимент, а оба слова "positive" и "negative" есть в словаре GPT.

В этом задании мы пойдём дальше и самостоятельно реализуем top-k сэмплирование, чтобы генерировать отзывы на фильмы, подобные тем, что в датасете IMDB.

Мы будем продолжать отзывы, используя в качестве "затравки" для модели начало реальных ревью.

### Top-k & Top-p sampling

Чтобы генерация текста была более разнообразной, мы будем использовать top k сэмплирование, которое мы изучали в этом юните. Его смысл в том, что на каждом шаге мы перед сэмплированием зануляем вероятности всех слов, кроме k самых вероятных.

In [ ]:
def topk_sample(scores, k):
    """
    Sample from logits using multinomial distribution
    Before sampling, all logits except k greatest are zeroed out.
    Args:
        scores: scores for every word in the vocabulary. Must be 1-dimensional: (num_words_in_vocab)
        k: int, how many hypotheses to sample from
    Returns:
        1 draw from dictribution, torch.LongTensor of shape (1)
    """
    assert k >= 1, 'k must be >=1'
    assert scores.ndim == 1, 'logits must have 1 dimension' 

    # TASK: implement top-k sampling
    # First, get top k values and theoir indices using torch.topk
    # 2nd, fill logits with some small value (e. g. 1e-6)
    # Next, move values back to their place. Note that you will find ellipsis (...) and None useful
    # Finally, use softmax to obtain probabilities and get 
    # Our implementation is 6 lines
    
    # YOUR CODE STARTS

    # YOUR CODE ENDS
    return predicted_ids

Проверка кода:

In [ ]:
logits = torch.randn(5)
for _ in range(10):
    res1 = topk_sample(logits, 1)
    res2  = topk_sample(logits, 1)
    assert res1.ndim == 0

    assert res1.equal(res2)
not_equal = False
for _ in range(10):
    if not topk_sample(logits, 2).equal(topk_sample(logits, 2)):
        not_equal = True
assert not_equal
print('OK!')

Также мы будем использовать top-p sampling, который мы уже проходили на лекциях.

In [ ]:
def topp_sample(scores, p):
    """
    Sample from logits using multinomial distribution
    Before sampling, all logits except those largest
    whose cumsum exceeds p are zeroed out
    Args:
        scores: scores for every word in the vocabulary. Must be 1-dimensional: (num_words_in_vocab)
        p: float, cumulative probability of most high-scored tokens
    Returns:
        1 draw from dictribution, torch.LongTensor of shape (1)
    """
    assert  0 < p < 1, 'p must be between 0 and 1'
    assert scores.ndim == 1, 'logits must have 1 dimension' 

    # TASK: implement top-p sampling
    # 1. sort scores with descending order
    # 2. get cumulative sum of softmaxed logits
    # 3. get indices when cumulative sum is larger than p. These indices should be zeroed out
    # 4. get real indices that should be zeroes out from sorted indices. Use tensor slicing
    # 5. fill these indices with some small value (e. g. -1e6)
    # 6. sample, as you did in top-k sampling
    # Our implementation is 7 lines
    
    # YOUR CODE STARTS

    # YOUR CODE ENDS
    return predicted_ids

Проверка кода:

In [ ]:
logits = torch.Tensor([-.1, -.2, .9])
for _ in range(10):
    res1 = topp_sample(logits, 0.8)
    res2  = topp_sample(logits, 0.8)
    assert res1.ndim == 0

    assert res1.equal(res2)
logits = torch.Tensor([.5, .5, .5])
not_equal = False
for _ in range(10):
    if not topp_sample(logits, 0.8).equal(topp_sample(logits, 0.8)):
        not_equal = True
assert not_equal
print('OK!')

Теперь напишите функцию генерации текста:

In [ ]:
def generate_text(model, ids, length, k=None, p=None):
    """
    Generate text with language model with ids as prompt
    Args:
        model: huggingface LM model
        ids: input ids, from tokenizer.encode, must be 2-dimensional
        length: int, how many tokens to geenrate
        k: int, how many hypotheses to sample from in top-k sampling
    Returns:
        token ids from generated text, together with token ids of prompts, 2d LongTensor
    """
    assert length >= 1
    if k and p:
        raise RuntimeError('Cannot use topk and topp sampling simultaneously')
    # TASK: write generation loop
    # For every timestamp:
    # - obtain model output
    # - get logits for last symbol
    # - apply topk sampling to get new index
    # - concatenate it to the ids to form new input
    # in the end, the `ids` tensor will have full generation
    # our implementation is 5 lines
    
    # YOUR CODE STARTS

    # YOUR CODE ENDS
    return ids.cpu().detach()

Проверка кода:

In [ ]:
ids = torch.randint(0, 1000, (4,)).to(device)
generation = generate_text(model, ids, 4, k=2)

assert generation.shape == torch.Size([8])

generation = generate_text(model, ids, 3, p=.5)

assert generation.shape == torch.Size([7])

### Запускаем генерацию

Зададим модели какой-нибудь prompt, например восторженный отзыв о фильме, и посмотрим, как она продолжит его. Так как модель сэмплит на каждом шаге, нам интересно посмотреть на несколько траекторий сразу. Для этого мы и писали батч-режим в генерации.

In [ ]:
prompt = 'What a lovely film !'

Подготовим входные данные для GPT

In [ ]:
ids = tokenizer.encode(prompt, return_tensors="pt").squeeze().to(device)

Запускаем генерацию! Можно запустить на 10-20 токенов, можно и больше

In [ ]:
n_tries = 5

In [ ]:
generations = [generate_text(model, ids, length=40, k=10).cpu().detach() for _ in range(n_tries)]

In [ ]:
for generation in generations:
    print(tokenizer.decode(generation))
    print('\n' + '=' * 40 + '\n')

А теперь запустим генерацию с top-p сэмплированием

In [ ]:
generations = [generate_text(model, ids, length=4, p=0.5).cpu().detach() for _ in range(n_tries)]
for generation in generations:
    print(tokenizer.decode(generation))
    print('\n' + '=' * 40 + '\n')

# Часть 2. Few-shot learning

Одной из особенностей GPT, про которую авторы написали, является способность к few-shot learning. В этой части домашнего задания мы на примере хорошо знакомой нам задачи классификации текстов исследуем, как хорошо модель умеет различать сентимент отзывов без дополнительного обучения.

## Датасет.
Мы будем снова использовать датасет imdb, на котором мы уже обучили несколько моделей. Так как обучать саму модель мы не будем (в few-shot парадигме веса не меняются), то можем сразу взять валидационную часть датасета.

In [ ]:
text_dataset = datasets.load_dataset("imdb")
texts = text_dataset["test"]["text"]
labels = text_dataset["test"]["label"]

### Метки для задачи классификации на естественном языке
Поскольку языковая модель знает только токены языка и не знает маппинга между индексами и лейблами, нам нужно будет сравнивать лейблы напрямую. С одной стороны, это создаёт дополнительные сложности: модель может предсказать "Positive Sentiment" вместо "positive", поэтому нам потребуется минимальный постпроцессинг, чтобы учесть большинство таких случаев.

In [ ]:
mapping = {idx: label for idx, label in enumerate(text_dataset["train"].features["label"].names)}
mapping

### Игрушечный датасет
Чтобы быстрее прототипировать наши вводы (prompts), полезно взять маленькую часть датасета и проверять гипотезы на ней. Так как imdb упорядочен по лейблам, а самих лейблов всего 2, мы берём одинаковое количество примеров с начала и с конца.

In [ ]:
def get_toy_dataset(dataset, split, length = 100):
    texts = dataset[split]["text"]
    labels = dataset[split]["label"]
    small_texts = texts[:length // 2] + texts[-length//2:]
    small_labels = labels[:length // 2] + labels[-length//2:]
    return small_texts, small_labels

In [ ]:
assert len(get_toy_dataset(text_dataset, "test", 100)[0]) == 100

## Few-shot learning

В данном задании мы будем не обучать модель на данных, а использовать уже обученную на огромном корпусе языковую модель, чтобы "угадывать" сентимент отзывов.

Модель будет видеть в качестве затравки (prompt) описание задачи на естественном языке.

Нам потребуется:
- описание задачи
- лейблы
- несколько (хотя бы 3) примеров
- вспомогательные маркеры (разделитель примеров и маркер конца входа)

Например:

```
Translate from English to French
sea otter => loutre de mer
peppermint => menthe poivrée
plush girafe  => 
```

Первая строка здесь -- описание задачи. Мы решаем задачу классификации, поэтому нам нужно
описать задачу примерно как `To which category does the text belong? positive , negative `

Вторая и третья строки -- примеры входа и выхода. В нашем случае примерами входа и выхода будут служить тексты ревью и метки классов positive и negative (а не 0 и 1, как раньше).

Наконец, последняя строка -- это тот текст, который модель должна классифицировать. Модель уже "видела" выше пример того, что после маркера конца входа идёт сентимент, и мы ожидаем, что она сможет выучить эту закономерность.

В данном задании часть входа будет написана за вас, вам надо будет придумать обучающие примеры для few-shot learning'а

In [ ]:
marker = ' => '
separator = '\n'

In [ ]:
labels = ["positive" , "negative"]
labels_text = " , ".join(labels)
task_description = f'To which category does the text belong?: {labels_text} ' + separator

In [ ]:
examples = [
    # TASK define examples
    # They should be a tuple with text and label
    # label should be from "labels"
    # Get 3-10 different examples, they should cover both positive 
    # and negative reviews
    # YOUR CODE STARTS

    # YOUR CODE ENDS
]

In [ ]:
assert len(examples) > 1, 'Write at least two different examples'
for ex in examples:
    assert ex[1] in labels

In [ ]:
input = task_description  + separator.join([marker.join(example) for example in examples])

In [ ]:
print(input)

Сравните получившийся инпут с примером выше. Мы получили то, что надо!

In [ ]:
n_chars = 50

In [ ]:
def get_prompt(beginning, separator, text, marker) -> str:
    return ' '.join([input,  separator, text[:n_chars] + text[-n_chars:], marker])

## Время проверить работу нашей модели на маленьком датасете!

In [ ]:
small_texts, small_labels = get_toy_dataset(text_dataset, "test")

Так как мы предсказываем сентимент текста, каждый из который задаётся одним токеном, то нам
- не надо сэмплировать. Мы хотим, чтобы модель честно учитывала вероятности для токенов
- достаточно сделать один шаг генерации. Модель должна будет предсказать лейбл текста, а он, как мы выяснили, однотокенный.

Таким образом, нам просто надо сделать один шаг жадного предсказания (greedy decoding).

In [ ]:
def predict_on_text(model, ids) -> int:
    """
    Run Language Model on text and use logits from last time 
    step to predict sentence category with greedy decoding
    Args:
        model: huggingface LM model
        ids: LongTensor . Text encoded with tokenizer.
    Returns:
        model prediction, index of most probable word
    """
    # TASK: run a model and get index most probable logit
    # HINT: do not forget to detach tensors
    # HINT 2: GPT-2 model returns tuple where logits are stored in the
    # 1st item. You do not need 2nd item at all
    # Our implementation is 2 lines
    # YOUR CODE STARTS

    # YOUR CODE ENDS
    return idx

Проверка кода:

In [ ]:
ids = tokenizer.encode('hello world !', return_tensors="pt",)
prediction = predict_on_text(model, ids.to(device))
assert isinstance(prediction, int)

In [ ]:
def predict_on_dataset(model, texts, target, mapping):
    """
    Run Language Model on a dataset and use its predictions as
    labels for sentences
    Args:
        model: huggingface LM model
        texts: List[str], texts of dataset
        target: List[int], indices of classes
        mapping: Dict[int, str] mapping from class indices to class labels
    Returns:
        model predictions, "as is", List[str]
        target labels, after mapping, List[str]
    """
    if len(texts) != len(target):
        raise RuntimeError('Texts and target lengths mismatch')
    # max_length has additional -1 because we will generate exactly 1 token 
    # with LM
    max_length = model.config.n_ctx - 1
    predictions = []
    labels = []
    for idx in tqdm(range(len(texts))):
        text, label = texts[idx], target[idx]
        ids = tokenizer.encode(
            get_prompt(input, separator, text, marker), 
            add_special_tokens=False, 
            return_tensors="pt",
            max_length=max_length
        )
        idx = predict_on_text(model, ids.to(device))
        predictions.append(tokenizer.decode([idx]))  # generation.cpu().numpy()[0][-1]
        labels.append(mapping[label]) 
    return predictions, labels

In [ ]:
predictions, target = predict_on_dataset(model, small_texts, small_labels, mapping)

### Замер качества

Посмотрим, что выдаёт нам языковая модель:

In [ ]:
print(predictions[:10])

Можно заметить, что предсказанные метки не совсем похожи на pos и neg, которые были у нас в таргете. Напишем простую функцию, которая нормализует предсказания модели

In [ ]:
def normalize_prediction(raw_output: str) -> str:
    """
    Helper that transforms model prediction to normal
    For example, ' positive' -> 'pos', ' Negative' -> neg
    Args:
        raw_output: str, output from language model
    Returns:
        normalized value: no leading/trailing spaces, lowercase,
        at most 3 chars long
    """
    # TASK: write prediction normalizer
    # You need to remove spaces, lowercase and get only 3 leading chars
    # Our implementation is 1 line
    # YOUR CODE STARTS

    # YOUR CODE ENDS
    return normalized_output

Напишем функцию, вычисляющую точность:

In [ ]:
def accuracy(predictions, labels) -> float:
    """
    Helper that transforms model prediction to normal
    For example, ' positive' -> 'pos', ' Negative' -> neg
    Args:
        predictions: List[str], model predictions, should be labels in natural language
        labels: List[str], target labels
    Returns:
        accuracy, float from [0, 1]
    """
    if not labels:
        # sanity check to avoid zero division
        return 0
    if len(predictions) != len(labels):
        raise ValueError(f'Predictions and labels have mismatched length')
    total = len(predictions)
    match = 0
    # TASK: calculate accuracy
    # For every pair of predicted and real labels, check that they coincide.
    # You can use zip() method for iteration over two sequences
    # simultaneously.
    # Our implementation is 3 lines
    # YOUR CODE STARTS

    # YOUR CODE ENDS
    return accuracy

In [ ]:
normalized_predictions = [normalize_prediction(label) for label in predictions]

In [ ]:
accuracy(normalized_predictions, target)

### Ура, модель без обучения работает лучше случайного угадывания!

Тем не менее, это заметно хуже, чем у полносвязной нейросети. **Учитывая требования по ресурсам и время 
работы, использование такой модели в реальных задачах непрактично!**

## А теперь замер на всем тестовом сете:

In [ ]:
predictions, target = predict_on_dataset(model, texts, labels, mapping)
normalized_predictions = [normalize_prediction(label) for label in predictions]
accuracy(normalized_predictions, target)

Все задание считается выполненным, если вы достигли accuracy 0.6 и выше на тестовом датасете. 

Данный пример призван продемонстрировать, что большая языковая модель хоть и справляется с задачей без лишних данных, но делает это заметно хуже специально созданных для задачи моделей, к тому же работает непростительно долго.

### Резюме

В этом задании мы применили большие языковые модели на практике и решили с её помощью 2 задачи:
- conditional генерация текста
- few-shot классификация текстов

Мы увидели, что модель генерирует связный текст и может решать задачу классификации без изменений своих весов.

Несмотря на возможности языковых моделей, их применение на практике всё ещё осложнено большими потребляемыми ресурсами, а также невозможностью интерпретировать предсказания.